# Dummy

In [15]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Lire les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Séparer les données d'entraînement
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")

# Préparer les colonnes numériques
numeric_cols = train.select_dtypes(include=['number']).columns

# Imputer les valeurs manquantes pour les colonnes numériques
imputer = SimpleImputer(strategy='median')
train_numeric = imputer.fit_transform(train[numeric_cols])
test_numeric = imputer.transform(app_test[numeric_cols])

# Normaliser les colonnes numériques
scaler = MinMaxScaler(feature_range=(0, 1))
train_numeric = scaler.fit_transform(train_numeric)
test_numeric = scaler.transform(test_numeric)

# Créer et entraîner le DummyClassifier
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(train_numeric, train_labels)

# Faire des prédictions sur les données d'entraînement (pour évaluation)
train_predictions = dummy_clf.predict(train_numeric)

# Calculer le score de précision sur les données d'entraînement
train_score = accuracy_score(train_labels, train_predictions)

# Faire des prédictions sur les données de test
dummy_predictions = dummy_clf.predict(test_numeric)

# Créer le DataFrame de soumission
submit = app_test[['SK_ID_CURR']].copy()
submit['TARGET'] = dummy_predictions

# Afficher les résultats
print("Score du modèle sur les données d'entraînement :", train_score)
print(submit.head())


Score du modèle sur les données d'entraînement : 0.8524786650376062
   SK_ID_CURR  TARGET
0      342180       0
1      259636       0
2      305882       0
3      243264       0
4      264946       0


# Regression logistique

In [17]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()

# Séparer les colonnes numériques et non numériques
numeric_cols = train.select_dtypes(include=['number']).columns
categorical_cols = train.select_dtypes(exclude=['number']).columns

# Imputation des valeurs manquantes pour les colonnes numériques
imputer = SimpleImputer(strategy='median')
train_numeric = imputer.fit_transform(train[numeric_cols])
test_numeric = imputer.transform(app_test[numeric_cols])

# Normalisation des colonnes numériques
scaler = MinMaxScaler(feature_range=(0, 1))
train_numeric = scaler.fit_transform(train_numeric)
test_numeric = scaler.transform(test_numeric)
train_final = pd.DataFrame(train_numeric, columns=numeric_cols)
test_final = pd.DataFrame(test_numeric, columns=numeric_cols)

# Créer et entraîner le modèle
log_reg = LogisticRegression(C=0.0001)
log_reg.fit(train_final, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = log_reg.predict_proba(train_final)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Faire des prédictions sur l'ensemble de test
log_reg_pred = log_reg.predict_proba(test_final)[:, 1]

# Créer le DataFrame de soumission
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()


ROC AUC Score: 0.6831


/var/folders/25/t6v1433n06x6vfl9wjbxn8sw0000gn/T/ipykernel_1249/2695516506.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['TARGET'] = log_reg_pred


,SK_ID_CURR,TARGET
0,342180,0.081685
1,259636,0.064739
2,305882,0.077060
3,243264,0.063606
4,264946,0.064830


# Arbre de décision

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Convertir toutes les colonnes en numériques en utilisant LabelEncoder
label_encoders = {}
for col in app_train.columns:
    if app_train[col].dtype == 'object':
        le = LabelEncoder()
        app_train[col] = le.fit_transform(app_train[col].astype(str))
        if col in app_test.columns:
            app_test[col] = le.transform(app_test[col].astype(str))
        label_encoders[col] = le

# Séparer les données d'entraînement et les labels
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()
    train_labels = None

# Préparer les données de test sans l'identifiant
test = app_test.drop(columns=['SK_ID_CURR'])

# S'assurer que les colonnes sont alignées avant l'imputation
common_cols = train.columns.intersection(test.columns)
train = train[common_cols]
test = test[common_cols]

# Imputer les valeurs manquantes avec la médiane
imputer = SimpleImputer(strategy='median')
train = imputer.fit_transform(train)
test = imputer.transform(test)

# Créer le modèle d'Arbre de Décision avec des hyperparamètres
decision_tree = DecisionTreeClassifier(max_depth=7, min_samples_split=10, min_samples_leaf=5)

# Entraîner le modèle
decision_tree.fit(train, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = decision_tree.predict_proba(train)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Extraire l'importance des caractéristiques
feature_importance_values = decision_tree.feature_importances_
feature_importances = pd.DataFrame({'feature': common_cols, 'importance': feature_importance_values})

# Faire des prédictions sur l'ensemble de test
predictions = decision_tree.predict_proba(test)[:, 1]

# Créer le DataFrame de soumission
submit = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': predictions})

print("Prédictions:")
print(submit.head())


ROC AUC Score: 0.7277
Prédictions:
   SK_ID_CURR    TARGET
0      342180  0.045878
1      259636  0.079470
2      305882  0.045878
3      243264  0.037365
4      264946  0.061666


# Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Convertir toutes les colonnes en numériques en utilisant LabelEncoder
label_encoders = {}
for col in app_train.columns:
    if app_train[col].dtype == 'object':
        le = LabelEncoder()
        app_train[col] = le.fit_transform(app_train[col].astype(str))
        if col in app_test.columns:
            app_test[col] = le.transform(app_test[col].astype(str))
        label_encoders[col] = le

# Séparer les données d'entraînement et les labels
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()
    train_labels = None

# Préparer les données de test sans l'identifiant
test = app_test.drop(columns=['SK_ID_CURR'])

# S'assurer que les colonnes sont alignées avant l'imputation
common_cols = train.columns.intersection(test.columns)
train = train[common_cols]
test = test[common_cols]

# Imputer les valeurs manquantes avec la médiane
imputer = SimpleImputer(strategy='median')
train = imputer.fit_transform(train)
test = imputer.transform(test)

# Créer le modèle RandomForest
random_forest = RandomForestClassifier(n_estimators=1, verbose=1, n_jobs=-1)

# Entraîner le modèle
random_forest.fit(train, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = random_forest.predict_proba(train)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Extraire l'importance des caractéristiques
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': common_cols, 'importance': feature_importance_values})

# Faire des prédictions sur l'ensemble de test
predictions = random_forest.predict_proba(test)[:, 1]

# Créer le DataFrame de soumission
submit = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': predictions})

print("Prédictions:")
print(submit.head())

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


ROC AUC Score: 0.8297
Prédictions:
   SK_ID_CURR  TARGET
0      342180     0.0
1      259636     0.0
2      305882     0.0
3      243264     0.0
4      264946     0.0


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


# Fonction unique

In [23]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import lightgbm as lgb
import logging
import sys

def process_and_predict(train_file, test_file):
    # Configurer le logging pour ignorer les messages d'information de LightGBM
    logging.getLogger('lightgbm').setLevel(logging.ERROR)
    
    # Rediriger stdout pour éviter les messages d'information de LightGBM
    original_stdout = sys.stdout
    sys.stdout = open('/dev/null', 'w')  # Sur Unix/Linux/Mac
    # sys.stdout = open(os.devnull, 'w')  # Sur Windows
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    # Créer et entraîner le DummyClassifier
    dummy_clf = DummyClassifier(strategy="stratified")
    dummy_clf.fit(train_final, train_labels)
    dummy_predictions = dummy_clf.predict(test_final)
    dummy_score = dummy_clf.score(train_final, train_labels)
    
    # Créer et entraîner le modèle LogisticRegression
    log_reg = LogisticRegression(C=0.0001, max_iter=1000)
    log_reg.fit(train_final, train_labels)
    train_pred_log_reg = log_reg.predict_proba(train_final)[:, 1]
    roc_auc_log_reg = roc_auc_score(train_labels, train_pred_log_reg)
    
    # Créer et entraîner le modèle DecisionTreeClassifier
    decision_tree = DecisionTreeClassifier(max_depth=7, min_samples_split=10, min_samples_leaf=5)
    decision_tree.fit(train_final, train_labels)
    train_pred_tree = decision_tree.predict_proba(train_final)[:, 1]
    roc_auc_tree = roc_auc_score(train_labels, train_pred_tree)
    
    # Créer et entraîner le modèle RandomForestClassifier
    random_forest = RandomForestClassifier(n_estimators=1, verbose=1, n_jobs=-1)
    random_forest.fit(train_final, train_labels)
    train_pred_rf = random_forest.predict_proba(train_final)[:, 1]
    roc_auc_rf = roc_auc_score(train_labels, train_pred_rf)
    
    # Créer et entraîner le modèle CatBoostClassifier
    catboost_model = CatBoostClassifier(iterations=100, depth=7, learning_rate=0.1, loss_function='Logloss', verbose=0)
    catboost_model.fit(train_final, train_labels)
    train_pred_catboost = catboost_model.predict_proba(train_final)[:, 1]
    roc_auc_catboost = roc_auc_score(train_labels, train_pred_catboost)
    catboost_predictions = catboost_model.predict_proba(test_final)[:, 1]
    
    # Créer et entraîner le modèle LightGBMClassifier
    lightgbm_model = lgb.LGBMClassifier(n_estimators=100, max_depth=7, learning_rate=0.1)
    lightgbm_model.fit(train_final, train_labels)
    train_pred_lightgbm = lightgbm_model.predict_proba(train_final)[:, 1]
    roc_auc_lightgbm = roc_auc_score(train_labels, train_pred_lightgbm)
    lightgbm_predictions = lightgbm_model.predict_proba(test_final)[:, 1]
    
    # Rétablir stdout
    sys.stdout = original_stdout
    
    # Afficher les résultats dans un tableau lisible
    results = {
        'Model': ['DummyClassifier', 'LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier', 'CatBoostClassifier', 'LightGBMClassifier'],
        'ROC AUC Score': [dummy_score, roc_auc_log_reg, roc_auc_tree, roc_auc_rf, roc_auc_catboost, roc_auc_lightgbm]
    }
    results_df = pd.DataFrame(results)
    
    print("Scores ROC AUC des modèles :")
    print(results_df.to_string(index=False))
    
    return {
        'DummyClassifier': dummy_predictions,
        'LogisticRegression': log_reg.predict_proba(test_final)[:, 1],
        'DecisionTreeClassifier': decision_tree.predict_proba(test_final)[:, 1],
        'RandomForestClassifier': random_forest.predict_proba(test_final)[:, 1],
        'CatBoostClassifier': catboost_predictions,
        'LightGBMClassifier': lightgbm_predictions
    }

# Appel de la fonction avec les chemins des fichiers
predictions = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Scores ROC AUC des modèles :
                 Model  ROC AUC Score
       DummyClassifier       0.852316
    LogisticRegression       0.685603
DecisionTreeClassifier       0.727685
RandomForestClassifier       0.830694
    CatBoostClassifier       0.768578
    LightGBMClassifier       0.798777


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


# Fonction unique avec CV

In [61]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import lightgbm as lgb
import logging
import sys
import time
from tabulate import tabulate

def measure_prediction_time(model, X, n_predictions=1000):
    """Mesurer le temps de prédiction moyen sur n_predictions échantillons."""
    start_time = time.time()
    for _ in range(n_predictions):
        _ = model.predict_proba(X)[:, 1]
    end_time = time.time()
    elapsed_time = end_time - start_time
    return "{:.2f} s".format(elapsed_time)

def process_and_predict(train_file, test_file):
    # Configurer le logging pour ignorer les messages d'information de LightGBM
    logging.getLogger('lightgbm').setLevel(logging.ERROR)
    
    # Rediriger stdout pour éviter les messages d'information de LightGBM
    original_stdout = sys.stdout
    sys.stdout = open('/dev/null', 'w')  # Sur Unix/Linux/Mac
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    # Créer et entraîner le DummyClassifier
    dummy_clf = DummyClassifier(strategy="stratified")
    dummy_clf.fit(train_final, train_labels)
    dummy_predictions = dummy_clf.predict_proba(test_final)[:, 1]
    dummy_auc = roc_auc_score(train_labels, dummy_clf.predict_proba(train_final)[:, 1])
    dummy_time = measure_prediction_time(dummy_clf, test_final)
    
    # Définir les paramètres pour GridSearchCV pour LogisticRegression
    log_reg_params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}
    log_reg = LogisticRegression(max_iter=2000)  # Augmenté à 2000
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='roc_auc')
    log_reg_grid.fit(train_final, train_labels)
    log_reg_auc = roc_auc_score(train_labels, log_reg_grid.predict_proba(train_final)[:, 1])
    log_reg_time = measure_prediction_time(log_reg_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour DecisionTreeClassifier
    tree_params = {'max_depth': [5, 7, 10], 'min_samples_split': [10, 20, 50]}
    decision_tree = DecisionTreeClassifier()
    tree_grid = GridSearchCV(decision_tree, tree_params, cv=5, scoring='roc_auc')
    tree_grid.fit(train_final, train_labels)
    tree_auc = roc_auc_score(train_labels, tree_grid.predict_proba(train_final)[:, 1])
    tree_time = measure_prediction_time(tree_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour RandomForestClassifier
    rf_params = {'n_estimators': [10, 50, 100], 'max_depth': [5, 7, 10]}
    random_forest = RandomForestClassifier(n_jobs=-1)
    rf_grid = GridSearchCV(random_forest, rf_params, cv=5, scoring='roc_auc')
    rf_grid.fit(train_final, train_labels)
    rf_auc = roc_auc_score(train_labels, rf_grid.predict_proba(train_final)[:, 1])
    rf_time = measure_prediction_time(rf_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour CatBoostClassifier
    catboost_params = {'depth': [6, 7, 8], 'learning_rate': [0.01, 0.1, 0.3]}
    catboost_model = CatBoostClassifier(iterations=100, loss_function='Logloss', verbose=0)
    catboost_grid = GridSearchCV(catboost_model, catboost_params, cv=5, scoring='roc_auc')
    catboost_grid.fit(train_final, train_labels)
    catboost_auc = roc_auc_score(train_labels, catboost_grid.predict_proba(train_final)[:, 1])
    catboost_predictions = catboost_grid.predict_proba(test_final)[:, 1]
    catboost_time = measure_prediction_time(catboost_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour LightGBMClassifier
    lightgbm_params = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [6, 7, 8],
        'num_leaves': [2**6 - 1, 2**7 - 1, 2**8 - 1]  # Ajusté pour respecter la condition
    }
    lightgbm_model = lgb.LGBMClassifier()
    lightgbm_grid = GridSearchCV(lightgbm_model, lightgbm_params, cv=5, scoring='roc_auc')
    lightgbm_grid.fit(train_final, train_labels)
    lightgbm_auc = roc_auc_score(train_labels, lightgbm_grid.predict_proba(train_final)[:, 1])
    lightgbm_predictions = lightgbm_grid.predict_proba(test_final)[:, 1]
    lightgbm_time = measure_prediction_time(lightgbm_grid, test_final)
    
    # Rétablir stdout
    sys.stdout = original_stdout

    # Afficher les meilleurs hyperparamètres pour chaque modèle
    best_params = {
        'LogisticRegression': log_reg_grid.best_params_,
        'DecisionTreeClassifier': tree_grid.best_params_,
        'RandomForestClassifier': rf_grid.best_params_,
        'CatBoostClassifier': catboost_grid.best_params_,
        'LightGBMClassifier': lightgbm_grid.best_params_
    }
    
    print("Meilleurs hyperparamètres pour chaque modèle :")
    for model_name, params in best_params.items():
        print(f"{model_name}: {params}")
    
    # Calcul des métriques pour chaque modèle
    models = {
        'DummyClassifier': dummy_clf,
        'LogisticRegression': log_reg_grid,
        'DecisionTreeClassifier': tree_grid,
        'RandomForestClassifier': rf_grid,
        'CatBoostClassifier': catboost_grid,
        'LightGBMClassifier': lightgbm_grid
    }

    metrics = []

    for name, model in models.items():
        train_pred = model.predict(train_final)
        auc_score = roc_auc_score(train_labels, model.predict_proba(train_final)[:, 1])
        precision = precision_score(train_labels, train_pred, zero_division=0)
        recall = recall_score(train_labels, train_pred, zero_division=0)
        f1 = f1_score(train_labels, train_pred, zero_division=0)
        cm = confusion_matrix(train_labels, train_pred)
        cm_str = f"TN: {cm[0, 0]}, FP: {cm[0, 1]}, FN: {cm[1, 0]}, TP: {cm[1, 1]}"
        time_metric = measure_prediction_time(model, test_final)
        metrics.append([
            name,
            f"{auc_score:.5f}",
            f"{precision:.5f}",
            f"{recall:.5f}",
            f"{f1:.5f}",
            cm_str,
            time_metric
        ])
    
    # Affichage des résultats
    headers = ['Model', 'ROC AUC', 'Precision', 'Recall', 'F1 Score', 'Confusion Matrix', 'Time for 1000 Predictions (s)']
    results_table = tabulate(metrics, headers, tablefmt='grid')
    
    print("Résultats des modèles :")
    print(results_table)
    
    return {
        'DummyClassifier': dummy_predictions,
        'LogisticRegression': log_reg_grid.predict_proba(test_final)[:, 1],
        'DecisionTreeClassifier': tree_grid.predict_proba(test_final)[:, 1],
        'RandomForestClassifier': rf_grid.predict_proba(test_final)[:, 1],
        'CatBoostClassifier': catboost_predictions,
        'LightGBMClassifier': lightgbm_predictions
    }

# Appel de la fonction avec les chemins des fichiers
predictions = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


Meilleurs hyperparamètres pour chaque modèle :
LogisticRegression: {'C': 10}
DecisionTreeClassifier: {'max_depth': 7, 'min_samples_split': 50}
RandomForestClassifier: {'max_depth': 10, 'n_estimators': 100}
CatBoostClassifier: {'depth': 6, 'learning_rate': 0.3}
LightGBMClassifier: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'num_leaves': 63}
Résultats des modèles :
+------------------------+-----------+-------------+----------+------------+--------------------------------------------+---------------------------------+
| Model                  |   ROC AUC |   Precision |   Recall |   F1 Score | Confusion Matrix                           | Time for 1000 Predictions (s)   |
+========================+===========+=============+==========+============+============================================+=================================+
| DummyClassifier        |   0.50101 |     0.08438 |  0.08472 |    0.08455 | TN: 182069, FP: 15896, FN: 15827, TP: 1465 | 1.15 s                     